In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm, trange
from math import sqrt
import pandas as pd
import numpy as np
import heapq
import random

In [2]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [3]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
test_matrix = np.zeros((n, m))
K = 50
y_ui = np.zeros((n, m))
for i in base.itertuples():
    if i[3]>=4:
        train_matrix[i[1] - 1, i[2] - 1] = i[3]

for i in test.itertuples():
    if i[3]>=4:
        test_matrix[i[1] - 1, i[2] - 1] = i[3]

# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i,j] >= 4:
            y_ui[i,j] = 1

In [4]:
# Item-Based OCCF
# Jaccard Index
skj = np.zeros((m,m))
for i in range(m):
    for j in range(m):
        if i!=j:
            uk = len(set(np.where(train_matrix[:,i]!=0)[0]).union(set(np.where(train_matrix[:,j]!=0)[0])))
            if uk != 0:
                skj[i,j] = len(set(np.where(train_matrix[:,i]!=0)[0]).intersection(set(np.where(train_matrix[:,j]!=0)[0])))/uk  
IOCCF = np.zeros((n,m))
for i in range(n):
    for j in range(m):
        idx = list(set(np.where(train_matrix[i,:]!=0)[0]).intersection(set(np.where(skj[j,:]!=0)[0])))
        ls = skj[j,idx]
        ls = sorted(ls,reverse=True)
        IOCCF[i,j] = sum(ls[:K])
rank_ioccf = list()
for i in range(n):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in range(m):
        if j in exclude:
            continue
        ls.append((j,IOCCF[i,j]))
    ls = sorted(ls, key=lambda x: x[1], reverse=True)
    rank_ioccf.append([ls[j][0] for j in range(len(ls))])

In [5]:
# User-Based OCCF
# Jaccard Index
swu = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if i!=j:
            uk = len(set(np.where(train_matrix[i,:]!=0)[0]).union(set(np.where(train_matrix[j,:]!=0)[0])))
            if uk != 0:
                swu[i,j] = len(set(np.where(train_matrix[i,:]!=0)[0]).intersection(set(np.where(train_matrix[j,:]!=0)[0])))/uk 
UOCCF = np.zeros((n,m))
for j in range(m):
    for i in range(n):
        idx = list(set(np.where(train_matrix[:,j]!=0)[0]).intersection(set(np.where(swu[i,:]!=0)[0])))
        ls = swu[i,idx]
        ls = sorted(ls,reverse=True)
        UOCCF[i,j] = sum(ls[:K])
rank_uoccf = list()
for i in range(n):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in range(m):
        if j in exclude:
            continue
        ls.append((j,UOCCF[i,j]))
    ls = sorted(ls, key=lambda x: x[1], reverse=True)
    rank_uoccf.append([ls[j][0] for j in range(len(ls))])

In [6]:
# Hybrid
r_hybrid = 0.5
HOCCF = r_hybrid*IOCCF+(1-r_hybrid)*UOCCF
rank_hoccf = list()
for i in range(n):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in range(m):
        if j in exclude:
            continue
        ls.append((j,HOCCF[i,j]))
    ls = sorted(ls, key=lambda x: x[1], reverse=True)
    rank_hoccf.append([ls[j][0] for j in range(len(ls))])

In [7]:
def PreK(rank, truth, k):
    prek = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        preuk = 0
        ure+=1
        for j in range(k):
            if rank[i][j] in ls:
                preuk+=1
        prek+=preuk/k
    return prek/ure

In [8]:
def RecK(rank, truth, k):
    reck = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        recuk = 0
        ure += 1
        for j in range(k):
            if rank[i][j] in ls:
                recuk += 1
        reck+=recuk/ts
    return reck/ure

In [9]:
print("Item-based Pre@5:"+str(PreK(rank_ioccf, test_matrix, 5)))
print("Item-based Rec@5:"+str(RecK(rank_ioccf, test_matrix, 5)))
print("User-based Pre@5:"+str(PreK(rank_uoccf, test_matrix, 5)))
print("User-based Rec@5:"+str(RecK(rank_uoccf, test_matrix, 5)))
print("Hybrid Pre@5:"+str(PreK(rank_hoccf, test_matrix, 5)))
print("Hybrid Rec@5:"+str(RecK(rank_hoccf, test_matrix, 5)))

Item-based Pre@5:0.3530701754385959
Item-based Rec@5:0.11469222808336291
User-based Pre@5:0.3916666666666662
User-based Rec@5:0.12628445234304111
Hybrid Pre@5:0.38508771929824487
Hybrid Rec@5:0.12863426765771127
